# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)
new_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [4]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [54]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [6]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [10]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {'n_factors':[20, 50, 100], 'reg_all': [0.02, 0.05, 0.1]}
gs_model = GridSearchCV(SVD, param_grid, cv=3, n_jobs=-1)
gs_model.fit(data)

In [14]:
# print out optimal parameters for SVD after GridSearch
print(gs_model.best_score)
print(gs_model.best_params)

{'rmse': 0.8734252395972583, 'mae': 0.672007493504634}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [17]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2)

In [21]:
# cross validating with KNNBasic
from surprise import accuracy
sim_pearson = {'name':'pearson', 'user_based':True}
basic_pearson = KNNBasic(sim_options=sim_pearson)
cv_KNN_basic = cross_validate(basic_pearson, data, n_jobs=-1)

In [24]:
# print out the average RMSE score for the test set
for i in cv_KNN_basic.items():
    print(i)
print('----------------------')
print(np.mean(cv_KNN_basic['test_rmse']))

('test_rmse', array([0.98101203, 0.96752974, 0.97378613, 0.97224481, 0.97334153]))
('test_mae', array([0.75816496, 0.74792863, 0.75070515, 0.75125224, 0.74971839]))
('fit_time', (0.6051545143127441, 0.6197600364685059, 0.574394702911377, 0.6792953014373779, 0.5529208183288574))
('test_time', (1.1165456771850586, 1.0770351886749268, 1.0677459239959717, 1.0126173496246338, 1.061889410018921))
----------------------
0.9735828503467221


In [28]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_KNN_baseline = cross_validate(knn_baseline, data, n_jobs=-1)

In [30]:
for i in cv_KNN_baseline.items():
    print(i)
print('-------------------')
print(np.mean(cv_KNN_baseline['test_rmse']))

('test_rmse', array([0.87303622, 0.87258003, 0.87506992, 0.87404119, 0.88402476]))
('test_mae', array([0.6683695 , 0.66647926, 0.67020669, 0.66664121, 0.6741594 ]))
('fit_time', (0.6270556449890137, 0.7115042209625244, 0.6656324863433838, 0.6412723064422607, 0.7300481796264648))
('test_time', (1.618253231048584, 1.563554286956787, 1.528463363647461, 1.459587574005127, 1.6230902671813965))
-------------------
0.8757504236201189


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [31]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [32]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [33]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [34]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0818095771934293, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [35]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      

In [41]:
# try out the new function here!
user_rating = movie_rater(df_movies, 4, 'Comedy')

      movieId                               title        genres
6271    47491  Adam's Apples (Adams æbler) (2005)  Comedy|Drama


KeyboardInterrupt: Interrupted by user

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [42]:
user_rating = [{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [58]:
## add the new ratings to the original ratings DataFrame
df_final = new_df.append(user_rating, ignore_index=True)
df_final[df_final['userId'] == 1000]

,userId,movieId,rating
100836,1000,55245,5
100837,1000,2491,4
100838,1000,4718,4
100839,1000,5990,3


In [62]:
# train a model using the new combined DataFrame
reader = Reader()
new_data = Dataset.load_from_df(df_final, reader)
new_dataset = new_data.build_full_trainset()
# print('Number of users: ', new_dataset.n_users, '\n')
# print('Number of items: ', new_dataset.n_items)

In [63]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(new_dataset)

In [64]:
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append((m_id, svd.predict(1000, m_id)[3]))

In [68]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key=lambda x: x[1], reverse=True)
ranked_movies[:5]

[(1204, 4.5922010538817375),
 (912, 4.555667071282791),
 (750, 4.55037602195376),
 (898, 4.541459907776784),
 (4973, 4.530295697736327)]

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [69]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for idx, rec in enumerate(user_ratings):
        title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
        print('Reccomendation #', idx+1, ': ', title, '\n')
        n -= 1
        if n ==0:
            break
            
recommended_movies(ranked_movies,df_movies,5)

Reccomendation # 1 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Reccomendation # 2 :  694    Casablanca (1942)
Name: title, dtype: object 

Reccomendation # 3 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Reccomendation # 4 :  680    Philadelphia Story, The (1940)
Name: title, dtype: object 

Reccomendation # 5 :  3622    Amelie (Fabuleux destin d'Amélie Poulain, Le) ...
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.